#### 1) Написать приложение, которое собирает основные новости с сайтов news.mail.ru, lenta.ru, yandex.news. Для парсинга использовать xpath. Структура данных должна содержать: название источника, наименование новости, ссылку на новость, дата публикации

In [133]:
from pprint import pprint
from lxml import html
import requests
from pymongo import MongoClient
from datetime import datetime

In [134]:
now = datetime.now()
now_data = now.strftime(" %Y-%m-%d %H:%M").split(' ')[0]
news = []

In [135]:
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36'}

In [ ]:
# yandex.news

In [136]:
response = requests.get('https://m.yandex.ru/news/', headers=header)
dom = html.fromstring(response.text)

In [138]:
blocks = dom.xpath("//td[contains(@class,'stories-set__item')]")

for block in blocks:
    item = {}
    name = block.xpath(".//a[contains(@class,'link_theme_black')]/text()")
    link = block.xpath(".//a[contains(@class,'link_theme_black')]/@href")
    source = block.xpath(".//div[contains(@class,'story__date')]/text()")
    
    item['name'] = ''.join(name)
    item['link'] = 'https://m.yandex.ru' + ''.join(link)
    item['source'] = ''.join(source).split(' ')[0] 
    item['data'] = ''.join(source).split(' ')[-1] + now_data
    news.append(item)

In [ ]:
# lenta.ru

In [139]:
response = requests.get('https://lenta.ru/', headers=header)
dom = html.fromstring(response.text)

In [140]:
blocks = dom.xpath("//section[contains(@class,'row b-top7-for-main js-top-seven')]/*/div[contains(@class,'item')]")

for block in blocks:
    item = {}
    name = block.xpath(".//a/text()")
    link = block.xpath(".//a/@href")
    time = block.xpath(".//time[contains(@class,'g-time')]/text()")
    
    item['name'] = ''.join(name).replace('\xa0', ' ')
    item['link'] = 'https://lenta.ru' + ''.join(link)
    item['source'] = 'lenta.ru' 
    item['data'] = ''.join(time) + now_data
    news.append(item)

In [ ]:
# news.mail.ru

In [141]:
response = requests.get('https://news.mail.ru/', headers=header)
dom = html.fromstring(response.text)

In [142]:
blocks = dom.xpath("//ul//li[contains(@class,'list__item')]")

for block in blocks:
    item = {}
    name = block.xpath("./a[contains(@class,'list__text')][1]/text()") 
    link = block.xpath("./a[contains(@class,'list__text')][1]/@href")
    
    if name == []:
        break;
    
    item['name'] = ''.join(name).replace('\xa0', ' ')
    item['link'] = 'https://news.mail.ru' + ''.join(link)
    
    response2 = requests.get(item['link'], headers=header)
    dom2 = html.fromstring(response2.text)
    
    source = dom2.xpath("//span[contains(@class,'note')]//a[contains(@class,'breadcrumbs__link')]/span[contains(@class,'link__text')]/text()")
    time = dom2.xpath("//span[contains(@class,'note')]//span[contains(@class,' js-ago')]/@datatime")
    
    item['source'] = ''.join(source)
    item['data'] = ''.join(time) + now_data
    news.append(item)
    

#### 2) Сложить все новости в БД

In [143]:
db_name = 'news_db'
client = MongoClient('mongodb://127.0.0.1:27017')
db = client[db_name]
db.news_db.insert_many(news)